In [ ]:
# import settings and functions
%run ./../imports.ipynb

In [ ]:
plot_base_2d = um_view + "/tutorials/fun-2/plot_base_2d"


Options
=======

[-load_file] if provided mesh is load from file

[-file_name] mesh name

[-base ainsworth, ainsworth_labatto, demkowicz, bernstein]

[-space h1, l2, hcurl, hdiv]

[-oder o] where is approximation order

In [ ]:
base = 'demkowicz'
space = 'hcurl'

base = 'ainsworth'
space = 'h1'

base = 'ainsworth'
space = 'l2'

order = 3

In [ ]:
!rm out_*
!{plot_base_2d} \
-base {base} \
-space {space} \
-order {order} \
-log_sl verbose  

In [ ]:
!convert.py out*

In [ ]:
vtks = !ls out*vtk
print(vtks)

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
def show_glyphs(params):
    out_to_vtk = !ls -c1 {params.show_file}*vtk
    last_file=out_to_vtk[0]

    p = pv.Plotter(notebook=True)

    mesh = pv.read(last_file[:-3] + "vtk")

    # Subsample the mesh
    # mesh = mesh.extract_points(every_n=10)  # Keep every 10th point

    # Add glyphs
    N = 10  # Change this to the desired value
    arrows = mesh.glyph(orient=params.show_field, scale=params.show_field, factor=0.3, tolerance=0.02)
    print(arrows.array_names)
    p.add_mesh(arrows, cmap=params.p_cmap, clim=params.clim, show_scalar_bar=params.show_scalar_bar)

    if params.show_edges:
        mesh=mesh.shrink(0.95)

    jupyter_backend='ipygany'

    p.add_mesh(mesh, show_scalar_bar=False, opacity=0.2, color='gray', smooth_shading=True)

    p.camera_position = params.camera_position

    p.enable_parallel_projection()
    p.enable_image_style()

    p.show(jupyter_backend=jupyter_backend)
    if params.p_save:
        p.save_graphic(params.p_save)

params.clim = [0,1]
params.p_cmap = "rainbow"
params.show_field = "U"
params.show_file = vtks[3][:-3]
params.show_scalar_bar = False
params.show_edges = False
# show_glyphs(params)

In [ ]:
for i in range(0, len(vtks)):
    params.show_file = vtks[i][:-3]
    params.p_save = "shape_"+ str(i) +".pdf"
    print(space)
    if space == 'hcurl':
        params.p_save = "shape_"+ space + "_" + str(i) +".pdf"
        show_glyphs(params)
    else: 
        params.show_field = "U"
        params.warp_field_scalar = "U" 
        params.component = 1
        # params.p_cmap = "PiYG"
        params.camera_position =  [
            (-2, -2, 3),
            (0.35, 0.35, 0),
            (0.25, 0.25, 1),
        ]

        zoom_factor = 0.4  # Adjust this value to your needs

        camera_position = params.camera_position

        # Calculate the new camera position
        new_camera_position = [
            camera_position[0][0] + (camera_position[1][0] - camera_position[0][0]) * zoom_factor,
            camera_position[0][1] + (camera_position[1][1] - camera_position[0][1]) * zoom_factor,
            camera_position[0][2] + (camera_position[1][2] - camera_position[0][2]) * zoom_factor,
        ]

        params.camera_position = [new_camera_position, camera_position[1], camera_position[2]]

        params.clim = [-1, 1]
        params.warp_factor = 0.5
        params.p_save = "shape_"+ space + "_" + str(i) +".pdf"
        show_results(params)